<a href="https://colab.research.google.com/github/RyanKelton/CorrectiveMaintenance/blob/main/CodeLlama7BInstruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Fri Oct  6 03:26:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip install git+https://github.com/huggingface/transformers.git@refs/pull/25740/head accelerate


  Cloning https://github.com/huggingface/transformers.git (to revision refs/pull/25740/head) to /tmp/pip-req-build-80jc6ksh
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-80jc6ksh
  Running command git fetch -q https://github.com/huggingface/transformers.git refs/pull/25740/head
  Running command git checkout -q c6c6daa3f07e753cff91a08c4294df4a6ea6227b
  Resolved https://github.com/huggingface/transformers.git to commit c6c6daa3f07e753cff91a08c4294df4a6ea6227b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.1 MB/s eta 0:00:00
 

In [2]:
from transformers import AutoTokenizer
import transformers
import torch


In [3]:

model = "codellama/CodeLlama-7b-Instruct-hf" #base model is "codellama/CodeLlama-7b-hf"



In [4]:
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

(…)ct-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)-Instruct-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

(…)-7b-Instruct-hf/resolve/main/config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [5]:
prompts_dict = {
    "default": "Provide answers in C++",
    "problem_understanding": "Please describe the problem or task in detail.",
    "algorithm_explanation": "Explain the high-level algorithm or approach you would use to solve this problem.",
    "code_structure": "Provide an outline of the code's structure, including functions, classes, and main logic.",
    "input_output_format": "Specify the expected input format and the desired output format.",
    "error_handling": "Explain how you would handle potential errors or exceptions in the code.",
    "pseudocode": "Provide a pseudocode representation of your solution.",
    "variable_naming_comments": "Suggest meaningful variable names and add comments to clarify your code.",
    "edge_cases": "Identify any edge cases or special scenarios that need to be considered.",
    "code_testing_debugging": "Describe your testing strategy and how you would debug potential issues in the code.",
    "optimization": "Discuss potential optimizations or efficiency improvements for your code."
}

system = prompts_dict["algorithm_explanation"]
user = "Write a function that asks the user for 5 integer inputs, and then outputs the sum of all 5 numbers"

prompt = f"<s><<SYS>>\n{system}\n<</SYS>>\n\n{user}"


In [6]:
sequences = pipeline(
    prompt,
    do_sample=True,
    top_k=10,
    temperature=0.1,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
    add_special_tokens=False
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [7]:
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: <s><<SYS>>
Explain the high-level algorithm or approach you would use to solve this problem.
<</SYS>>

Write a function that asks the user for 5 integer inputs, and then outputs the sum of all 5 numbers.

<details><summary>Answer</summary>

```
def sum_of_5_numbers():
    numbers = []
    for i in range(5):
        number = int(input("Enter a number: "))
        numbers.append(number)
    return sum(numbers)
```
</details>

<details><summary>Example usage</summary>

```
Enter a number: 1
Enter a number: 2
Enter a number: 3
Enter a number: 4
Enter a number: 5
The sum of the 5 numbers is: 15
```
</details>

<


In [ ]:
#default

<</SOLUTION>>

#include <iostream>

using namespace std;

int main() {
    int num1, num2, num3;
    cout << "Enter three numbers: ";
    cin >> num1 >> num2 >> num3;
    int sum = num1 + num2 + num3;
    cout << "The sum of the three numbers is: " << sum << endl;
    return 0;
}

<</SOLUTION>>